In [ ]:
# 词语解析
import bz2
from urllib import parse
w = 0
with bz2.open('.\\sgns.target.word-character.char1-2.dynwin5.thr10.neg5.dim300.iter5.bz2', mode = 'rb') as f:
   while True:
        text = f.readline()
        if not text:
            break
        w += 1
        s = str(text).split(' ')[0][2:]
        s = s.encode()
        ss = s.decode().replace('\\x', '%')
        un = parse.unquote(ss)
        print(un)
print(w)        

In [1]:
# 加载预训练词向量
import pickle
from gensim.models import KeyedVectors

wv_from_text = KeyedVectors.load_word2vec_format('.\\sgns.target.word-character.char1-2.dynwin5.thr10.neg5.dim300.iter5.bz2',
    binary=False, encoding="utf8",  unicode_errors='ignore')

vocab = list(wv_from_text.vocab)
with open('..\\ETM-市长电话文本数据\\pred_embedding_baidu\\vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)
    
vector = wv_from_text.vectors
with open('..\\ETM-市长电话文本数据\\pred_embedding_baidu\\vector.pkl', 'wb') as f:
    pickle.dump(vector, f)

In [2]:
# 查看预训练词向量效果（不考虑模长）
word2id = dict([(word, i) for i,word in enumerate(vocab)])
id2word = dict([(i, word) for i,word in enumerate(vocab)])

words = ['烧烤','供暖','保险','交通','服务','医保','消防']
for word in words:
    if word in vocab:
        embedding_word = vector[word2id[word]]
        rank = vector.dot(embedding_word)
        mostSimilar = []
        [mostSimilar.append(idx) for idx in rank.argsort()[::-1]]
        nearest_neighbors = mostSimilar[:21]
        print('给定词:{}  近邻词:{}'.format(
                    word, [id2word[idx] for idx in nearest_neighbors][1:]))
        
"""
给定词:烧烤  近邻词:['烧烤炉', '烤全羊', '野炊', '烤肉', '铁板烧', '烤制', '风味餐厅', '烤羊肉串', '滑草', '农家饭']
给定词:供暖  近邻词:['采暖', '供热', '集中供热', '供暖系统', '采暖系统', '热泵', '源热泵', '供冷', '地板采暖', '暖通']
给定词:保险  近邻词:['保险机构', '保险', '保险金额', '人身保险', '附加险', '保险合同', '责任保险', '再保险', '商业保险', '被保险人']
给定词:交通  近邻词:['交通网络', '公路交通', '交通枢纽', '水陆交通', '交通条件', '交通运输', '轨道交通', '交通状况', '空中交通', '道路交通']
给定词:服务  近邻词:['应用服务', '服务提供者', '用户服务', '服务类型', '互联网服务', '服务', '服务供应商', '服务程序', '客户服务', '代理服务']
给定词:医保  近邻词:['非处方药', '非处方', '定点医院', '医疗保险', '城镇职工', 'ｍｌ', '工伤保险', '新农合', '失业保险', '参保']
给定词:消防  近邻词:['消防人员', '消防系统', '消防车', '消防器材', '消防安全', '公安消防部队', '消火栓', '消防设施', '灭火剂', '公安消防']

预训练词向量OK!
"""

给定词:沙尘暴  近邻词:['沙漠化', '尘暴', '沙暴', '沙化', '荒漠化', '沙尘', '酸雨', '天气现象', '雷雨大风', '灾害性', '干热风', '雪灾', '冰雹', '风沙', '厄尔尼诺现象', 'すなあらし', '┇', '土壤侵蚀', '泥石流', '风暴潮']


"\n给定词:烧烤  近邻词:['烧烤炉', '烤全羊', '野炊', '烤肉', '铁板烧', '烤制', '风味餐厅', '烤羊肉串', '滑草', '农家饭']\n给定词:供暖  近邻词:['采暖', '供热', '集中供热', '供暖系统', '采暖系统', '热泵', '源热泵', '供冷', '地板采暖', '暖通']\n给定词:保险  近邻词:['保险机构', '保险', '保险金额', '人身保险', '附加险', '保险合同', '责任保险', '再保险', '商业保险', '被保险人']\n给定词:交通  近邻词:['交通网络', '公路交通', '交通枢纽', '水陆交通', '交通条件', '交通运输', '轨道交通', '交通状况', '空中交通', '道路交通']\n给定词:服务  近邻词:['应用服务', '服务提供者', '用户服务', '服务类型', '互联网服务', '服务', '服务供应商', '服务程序', '客户服务', '代理服务']\n给定词:医保  近邻词:['非处方药', '非处方', '定点医院', '医疗保险', '城镇职工', 'ｍｌ', '工伤保险', '新农合', '失业保险', '参保']\n给定词:消防  近邻词:['消防人员', '消防系统', '消防车', '消防器材', '消防安全', '公安消防部队', '消火栓', '消防设施', '灭火剂', '公安消防']\n\n预训练词向量OK!\n"

In [2]:
# 查看预训练词向量效果（考虑模长）
import numpy as np
word2id = dict([(word, i) for i,word in enumerate(vocab)])
id2word = dict([(i, word) for i,word in enumerate(vocab)])

words = ['烧烤','供暖','保险','交通','服务','医保','消防']
for word in words:
    if word in vocab:
        embedding_word = vector[word2id[word]]
        rank = vector.dot(embedding_word)
        denom = embedding_word.T.dot(embedding_word).squeeze()
        denom = denom * np.sum(vector**2, 1)
        denom = np.sqrt(denom)
        rank = rank / denom
        mostSimilar = []
        [mostSimilar.append(idx) for idx in rank.argsort()[::-1]]
        nearest_neighbors = mostSimilar[:21]
        print('给定词:{}  近邻词:{}'.format(
                    word, [id2word[idx] for idx in nearest_neighbors][1:]))
        
"""
给定词:烧烤  近邻词:['烧烤炉', '烤全羊', '野炊', '烤肉', '铁板烧', '烤制', '风味餐厅', '烤羊肉串', '滑草', '农家饭']
给定词:供暖  近邻词:['采暖', '供热', '集中供热', '供暖系统', '采暖系统', '热泵', '源热泵', '供冷', '地板采暖', '暖通']
给定词:保险  近邻词:['保险机构', '保险', '保险金额', '人身保险', '附加险', '保险合同', '责任保险', '再保险', '商业保险', '被保险人']
给定词:交通  近邻词:['交通网络', '公路交通', '交通枢纽', '水陆交通', '交通条件', '交通运输', '轨道交通', '交通状况', '空中交通', '道路交通']
给定词:服务  近邻词:['应用服务', '服务提供者', '用户服务', '服务类型', '互联网服务', '服务', '服务供应商', '服务程序', '客户服务', '代理服务']
给定词:医保  近邻词:['非处方药', '非处方', '定点医院', '医疗保险', '城镇职工', 'ｍｌ', '工伤保险', '新农合', '失业保险', '参保']
给定词:消防  近邻词:['消防人员', '消防系统', '消防车', '消防器材', '消防安全', '公安消防部队', '消火栓', '消防设施', '灭火剂', '公安消防']

预训练词向量OK!
"""

给定词:烧烤  近邻词:['烤肉', '烧烤摊', '烧烤炉', '烧烤车', '烤', '烧烤架', '烤串', '野炊', '篝火', '火锅', '铁板烧', '海鲜', '烤肉酱', '烤制', '烤肉串', '烧烤店', '电炸锅', '烤肉架', '叉烧饭', '烤羊']
给定词:供暖  近邻词:['供热', '采暖', '供暖系统', '供冷', '集中供热', '采暖费', '取暖', '电采暖', '供热站', '供水', '换热站', '电热水', '土暖气', '采暖系统', '供热量', '地板采暖', '源热泵', '采暖设备', '热泵式', '采暖期']
给定词:保险  近邻词:['保险机构', '保险公司', '财产保险', '险种', '保险卡', '投保', '保险业务', '个人保险', '保险锁', '保险业', '人身保险', '保险处', '综合险', '财险', '财产险', '损失险', '保险期', '承保', '人寿保险', '保费']
给定词:交通  近邻词:['交通运输', '公路交通', '交通枢纽', '交通网络', '水陆交通', '交通中枢', '交通条件', '便利', '交通状况', '交通网', '道路交通', '北方交通', '交通车', '公共交通', '城市交通', '杭衢', '交通线路', '交通岛', '交通要道', '交通设施']
给定词:服务  近邻词:['服务商', '服务于', '网络服务', '优质服务', '咨询服务', '服务项目', '客户服务', '服务质量', '服务体系', '用户服务', '特服', '服务年', '服务提供商', '业务', '服务平台', '应用服务', '客户服务器', '域名服务', '乐机', '售后服务']
给定词:医保  近邻词:['医疗保险', '医保局', '非处方药', '非处方', '医保卡', '定点医院', '医保中心', '平保', '非处方药品', '城镇职工', '新农合', '定点药店', '生产厂家', '参保', '参保者', '农保', '医疗卡', '医疗险', '医疗', '医嘱']
给定词:消防  近邻词:['灭火', '消防泵', '消防艇', '消防安全', '消防系统', '消防人员', '公安

"\n给定词:烧烤  近邻词:['烧烤炉', '烤全羊', '野炊', '烤肉', '铁板烧', '烤制', '风味餐厅', '烤羊肉串', '滑草', '农家饭']\n给定词:供暖  近邻词:['采暖', '供热', '集中供热', '供暖系统', '采暖系统', '热泵', '源热泵', '供冷', '地板采暖', '暖通']\n给定词:保险  近邻词:['保险机构', '保险', '保险金额', '人身保险', '附加险', '保险合同', '责任保险', '再保险', '商业保险', '被保险人']\n给定词:交通  近邻词:['交通网络', '公路交通', '交通枢纽', '水陆交通', '交通条件', '交通运输', '轨道交通', '交通状况', '空中交通', '道路交通']\n给定词:服务  近邻词:['应用服务', '服务提供者', '用户服务', '服务类型', '互联网服务', '服务', '服务供应商', '服务程序', '客户服务', '代理服务']\n给定词:医保  近邻词:['非处方药', '非处方', '定点医院', '医疗保险', '城镇职工', 'ｍｌ', '工伤保险', '新农合', '失业保险', '参保']\n给定词:消防  近邻词:['消防人员', '消防系统', '消防车', '消防器材', '消防安全', '公安消防部队', '消火栓', '消防设施', '灭火剂', '公安消防']\n\n预训练词向量OK!\n"